In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


#### Import dependencies

In [ ]:
pip install mcfly

In [ ]:
import os
import statistics
import csv
import itertools
import tensorflow as tf
import keras
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import mcfly
import os
import statistics
import csv
import itertools
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sn
from numpy import mean, std

### PAMAP2

In [ ]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [ ]:
data = pd.read_pickle('pamap2.df')

#### LOSO split

In [ ]:
test_data = data[(data["subject_id"] == 102)]
train_data = data[(data["subject_id"] != 102) & (data["subject_id"] != 103)]
val_data = data[(data["subject_id"] == 103)]

In [ ]:
def fix_data(data):
    data = data.drop(data[data['activity_id']==0].index)
    data = data.interpolate()
    for colName in data.columns:
        if data[colName].dtype != 'object':
            data[colName] = data[colName].fillna(data[colName].mean())
    return data

train_data = fix_data(train_data)
test_data = fix_data(test_data)
val_data = fix_data(val_data)

#### Create sliding windows from samples

In [ ]:
df = data
window_size = 200
stride = 50

train_frames = []
for i in tqdm(range(0, len(train_data)-window_size, stride)):
    window = train_data.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      train_frames.append(window)

test_frames = []
for i in tqdm(range(0, len(test_data)-window_size, stride)):
    window = test_data.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      test_frames.append(window)

val_frames = []
for i in tqdm(range(0, len(val_data)-window_size, stride)):
    window = val_data.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      val_frames.append(window)

<ipython-input-8-93250bdbd828>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(train_data)-window_size, stride)):


  0%|          | 0/29972 [00:00<?, ?it/s]

<ipython-input-8-93250bdbd828>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(test_data)-window_size, stride)):


  0%|          | 0/5263 [00:00<?, ?it/s]

<ipython-input-8-93250bdbd828>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(val_data)-window_size, stride)):


  0%|          | 0/3483 [00:00<?, ?it/s]

In [ ]:
X_train_list = []
y_train_list = []
relevant_columns = [col for col in train_frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

#for each frame replace label with activity
for frame in tqdm(train_frames):
    X_train_list.append(frame[relevant_columns].values)
    y_train_list.append(frame.iloc[0]['activity_id'])

X_test_list = []
y_test_list = []
relevant_columns = [col for col in test_frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

for frame in tqdm(test_frames):
    X_test_list.append(frame[relevant_columns].values)
    y_test_list.append(frame.iloc[0]['activity_id'])

X_val_list = []
y_val_list = []
relevant_columns = [col for col in val_frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

for frame in tqdm(val_frames):
    X_val_list.append(frame[relevant_columns].values)
    y_val_list.append(frame.iloc[0]['activity_id'])

<ipython-input-9-ad4c13533411>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(train_frames):


  0%|          | 0/29653 [00:00<?, ?it/s]

<ipython-input-9-ad4c13533411>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(test_frames):


  0%|          | 0/5211 [00:00<?, ?it/s]

<ipython-input-9-ad4c13533411>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(val_frames):


  0%|          | 0/3443 [00:00<?, ?it/s]

In [ ]:
X_train = np.array(X_train_list)
y_train = np.array(to_categorical(y_train_list))

X_test = np.array(X_test_list)
y_test = np.array(to_categorical(y_test_list))

X_val = np.array(X_val_list)
y_val = np.array(to_categorical(y_val_list, num_classes = 25))

In [ ]:
print("X_train: ",X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

print("Y_train: ",y_train.shape)
print("Y_val: ",y_val.shape)
print("Y_test: ",y_test.shape)

X_train:  (29653, 200, 21)
X_val:  (3443, 200, 21)
X_test:  (5211, 200, 21)
Y_train:  (29653, 25)
Y_val:  (3443, 25)
Y_test:  (5211, 25)


In [ ]:
#fit scaler on train
scaler = StandardScaler()
train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)

In [ ]:
#transform test, val based on train scaler
val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(val_reshaped)
X_val = X_val_scaled.reshape(X_val.shape)

test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(test_reshaped)
X_test = X_test_scaled.reshape(X_test.shape)

In [ ]:
#n_steps, n_length,n_features = 4,50,19
n_steps, n_length,n_features = 4,50,21
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
valX = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [ ]:
from scipy.stats import entropy

js_divergences = np.zeros(trainX.shape[1])
for i in range(trainX.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(trainX[:, i])), len(np.unique(testX[:, i])))
    p, _ = np.histogram(trainX[:, i], bins=bins, density=True)
    q, _ = np.histogram(testX[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.6188317710272156


### MHEALTH

In [ ]:
cd "/content/drive/MyDrive/MHEALTHDATASET/"

/content/drive/MyDrive/MHEALTHDATASET


In [ ]:
df_mhealth = pd.read_pickle('mhealth_cnn.pkl')

#### LOSO split

In [ ]:
train_data = df_mhealth[(df_mhealth["subject"] != 'subject2') & (df_mhealth["subject"] != 'subject3')]
#train_data = df_mhealth[(df_mhealth["subject"] != 'subject4')]
test_data = df_mhealth[(df_mhealth["subject"] == 'subject2')]
val_data = df_mhealth[(df_mhealth["subject"] == 'subject3')]

In [ ]:
window_size = 200
stride = 50

train_frames = []
for i in tqdm(range(0, len(train_data)-window_size, stride)):
    window = train_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      train_frames.append(window)

test_frames = []
for i in tqdm(range(0, len(test_data)-window_size, stride)):
    window = test_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      test_frames.append(window)

val_frames = []
for i in tqdm(range(0, len(val_data)-window_size, stride)):
    window = val_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      val_frames.append(window)

<ipython-input-23-b0637a775a85>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(train_data)-window_size, stride)):


  0%|          | 0/5934 [00:00<?, ?it/s]

<ipython-input-23-b0637a775a85>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(test_data)-window_size, stride)):


  0%|          | 0/769 [00:00<?, ?it/s]

<ipython-input-23-b0637a775a85>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(val_data)-window_size, stride)):


  0%|          | 0/766 [00:00<?, ?it/s]

In [ ]:
X_train_list = []
y_train_list = []
relevant_columns = [col for col in train_frames[0].columns if col not in ['subject', 'activity']]

#for each frame replace label with activity
for frame in tqdm(train_frames):
    X_train_list.append(frame[relevant_columns].values)
    y_train_list.append(frame.iloc[0]['activity'])

X_test_list = []
y_test_list = []
for frame in tqdm(test_frames):
    X_test_list.append(frame[relevant_columns].values)
    y_test_list.append(frame.iloc[0]['activity'])

X_val_list = []
y_val_list = []
for frame in tqdm(val_frames):
    X_val_list.append(frame[relevant_columns].values)
    y_val_list.append(frame.iloc[0]['activity'])

<ipython-input-24-6f01deab48dd>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(train_frames):


  0%|          | 0/5564 [00:00<?, ?it/s]

<ipython-input-24-6f01deab48dd>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(test_frames):


  0%|          | 0/721 [00:00<?, ?it/s]

<ipython-input-24-6f01deab48dd>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(val_frames):


  0%|          | 0/718 [00:00<?, ?it/s]

In [ ]:
X_train = np.array(X_train_list)
y_train = np.array(to_categorical(y_train_list))

X_test = np.array(X_test_list)
y_test = np.array(to_categorical(y_test_list))

X_val = np.array(X_val_list)
y_val = np.array(to_categorical(y_val_list))

scale

In [ ]:
#fit scaler on train
scaler = StandardScaler()
train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)

In [ ]:
val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(val_reshaped)
X_val = X_val_scaled.reshape(X_val.shape)

test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(test_reshaped)
X_test = X_test_scaled.reshape(X_test.shape)

In [ ]:
print("X_train: ",X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

print("Y_train: ",y_train.shape)
print("Y_val: ",y_val.shape)
print("Y_test: ",y_test.shape)

X_train:  (5564, 200, 23)
X_val:  (718, 200, 23)
X_test:  (721, 200, 23)
Y_train:  (5564, 13)
Y_val:  (718, 13)
Y_test:  (721, 13)


In [ ]:
n_steps, n_length,n_features = 4,50,23
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
valX = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [ ]:
history = model.fit(trainX,y_train, epochs=8, validation_data=(valX, y_val), verbose=1, batch_size = 128)

Epoch 1/8
145/145 [==============================] - 50s 274ms/step - loss: 0.9927 - acc: 0.7386 - val_loss: 0.7552 - val_acc: 0.7479
Epoch 2/8
145/145 [==============================] - 27s 184ms/step - loss: 0.4170 - acc: 0.8370 - val_loss: 0.6233 - val_acc: 0.8038
Epoch 3/8
145/145 [==============================] - 26s 180ms/step - loss: 0.2493 - acc: 0.9037 - val_loss: 0.6133 - val_acc: 0.7982
Epoch 4/8
145/145 [==============================] - 26s 179ms/step - loss: 0.2131 - acc: 0.9163 - val_loss: 0.6280 - val_acc: 0.7918
Epoch 5/8
145/145 [==============================] - 25s 170ms/step - loss: 0.1770 - acc: 0.9322 - val_loss: 0.4989 - val_acc: 0.8528
Epoch 6/8
145/145 [==============================] - 26s 180ms/step - loss: 0.1558 - acc: 0.9383 - val_loss: 0.5336 - val_acc: 0.8537
Epoch 7/8
145/145 [==============================] - 26s 180ms/step - loss: 0.1406 - acc: 0.9460 - val_loss: 0.7195 - val_acc: 0.8242
Epoch 8/8
145/145 [==============================] - 26s 179ms

In [ ]:
from scipy.stats import entropy

js_divergences = np.zeros(trainX.shape[1])
for i in range(trainX.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(trainX[:, i])), len(np.unique(testX[:, i])))
    p, _ = np.histogram(trainX[:, i], bins=bins, density=True)
    q, _ = np.histogram(testX[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.36793205603952645


### ML model

In [ ]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=96, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Dropout(0.06)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64 , return_sequences= True ))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation = 'softmax'))

In [ ]:
plot_model(model, to_file='model1_plot.png', show_shapes=True,rankdir = 'TB',expand_nested = False,  show_layer_names=False, show_dtype =False)

In [ ]:
model.compile(Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['acc'])

In [ ]:
history = model.fit(trainX,y_train, epochs=8, validation_data=(valX, y_val), verbose=1 , batch_size  = 128 )

Epoch 1/8
232/232 [==============================] - 49s 190ms/step - loss: 0.4668 - acc: 0.8857 - val_loss: 0.2687 - val_acc: 0.9012
Epoch 2/8
232/232 [==============================] - 41s 175ms/step - loss: 0.0869 - acc: 0.9727 - val_loss: 0.4558 - val_acc: 0.8391
Epoch 3/8
232/232 [==============================] - 44s 188ms/step - loss: 0.0493 - acc: 0.9841 - val_loss: 0.4864 - val_acc: 0.8132
Epoch 4/8
232/232 [==============================] - 39s 170ms/step - loss: 0.0330 - acc: 0.9899 - val_loss: 0.5324 - val_acc: 0.8594
Epoch 5/8
232/232 [==============================] - 38s 164ms/step - loss: 0.0366 - acc: 0.9892 - val_loss: 0.6040 - val_acc: 0.8400
Epoch 6/8
232/232 [==============================] - 39s 170ms/step - loss: 0.0246 - acc: 0.9925 - val_loss: 0.8444 - val_acc: 0.7947
Epoch 7/8
232/232 [==============================] - 40s 171ms/step - loss: 0.0221 - acc: 0.9931 - val_loss: 0.6769 - val_acc: 0.8423
Epoch 8/8
232/232 [==============================] - 40s 174ms

In [ ]:
model.save("cnn_loso.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### predict on test

In [ ]:
loaded_model = tf.keras.models.load_model("cnn_loso.h5")

In [ ]:
lstm_y_test_pred = loaded_model.predict(testX)
y_test_argmax = y_test.argmax(axis=1)
y_test_pred_argmax = lstm_y_test_pred.argmax(axis=1)
og_test = np.argmax(y_test, axis=1)

163/163 [==============================] - 4s 14ms/step


In [ ]:
print("Smartphone Results with CNN-GRU for Test Set\n\n" + classification_report(y_test_pred_argmax,og_test , digits= 4))

Smartphone Results with CNN-GRU for Test Set

              precision    recall  f1-score   support

           1     0.9527    1.0000    0.9758       443
           2     0.9187    0.5773    0.7091       705
           3     0.1831    0.9789    0.3085        95
           4     0.8949    0.9249    0.9097       626
           5     0.3204    0.3277    0.3240       177
           6     1.0000    0.6376    0.7787       781
           7     0.4390    0.9885    0.6080       262
          12     0.7426    0.7795    0.7606       322
          13     0.1919    0.9661    0.3202        59
          16     1.0000    0.8167    0.8991       502
          17     1.0000    0.7681    0.8688       746
          24     0.9923    0.5254    0.6870       493

    accuracy                         0.7459      5211
   macro avg     0.7196    0.7742    0.6791      5211
weighted avg     0.8804    0.7459    0.7787      5211

